In [1]:
from fastai import *
from fastai.text import * 

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
df = pd.read_csv('merged.csv', sep=r'\<\|\>', header=None, names=['text', 'author'])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [5]:
df.head()

,text,author
0,"""If it isn’t one thing it’s the other,"" said t...",anon
1,"""Sour,"" said the fox about the rowanberries.",anon
2,'tis in vain to kick after you have once put o...,anon
3,An ill won penny will cast down a pound.,anon
4,A bad beginning makes a bad ending.,anon


In [6]:
df['text'] = df['text'].str.lower()

In [7]:
valid_pct = 0.05 #validation percent
df = df.iloc[np.random.permutation(len(df))]
cut = int(valid_pct * len(df)) + 1
train_df, valid_df = df[cut:], df[:cut]

In [8]:
len(train_df), len(valid_df)

(4575, 241)

In [9]:
nan_rows = df[df['text'].isnull()]

In [10]:
nan_rows

,text,author


In [11]:
data_lm = TextLMDataBunch.from_df('data', train_df, valid_df, text_cols='text',
                           label_cols='author')

In [12]:
# data_clas = TextClasDataBunch.from_df('data', train_df, valid_df, text_cols='text', label_cols='author', vocab=data_lm.train_ds.vocab, bs=32)

In [13]:
learn = language_model_learner(data_lm, pretrained_model=URLs.WT103, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy


RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 96 and 86 in dimension 1 at /Users/administrator/nightlies/pytorch-1.0.0/wheel_build_dirs/wheel_3.6/pytorch/aten/src/TH/generic/THTensorMoreMath.cpp:1333

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

In [ ]:
wd=1e-7
lr=1e-3
lrs = lr

In [ ]:
learn.fit(10,lrs, wd)

In [ ]:
learn.predict("xxbos", n_words=50, temperature=0.75)

In [ ]:
number_of_ideas = 100
ideas_counter = 0
all_ideas = []

for i in range(1000):
    idea = learn.predict("xxbos xxfld 1", n_words=20, temperature=0.8)
    ideas = idea.split("xxbos xxfld 1")
    ideas = ideas[1:-1]
    
    for idea in ideas:
        idea = idea.replace("xxbos xxfld 1 ","").strip()
        if(idea):
            all_ideas.append(idea)
            ideas_counter = ideas_counter+1
            
    if ideas_counter > number_of_ideas:
        break

In [ ]:
all_ideas

In [ ]:
learn.save_encoder('ft_enc')

In [ ]:
train_df.to_pickle('data/train_df.pkl')

In [ ]:
valid_df.to_pickle('data/valid_df.pkl')